### Camara estereoscopica
### Evaluación nube de puntos
##### Gibrán Zazueta

In [1]:
import open3d as o3d
import numpy as np
import copy
import time
import os
import sys
import glob
import yaml
import pyvista as pv
import pymeshfix as mf

from dictionary_manager import get_serial_dictionary

# Directory  
directory = "009"
patient = 'carlos-rivas'

# Parent Directory path  
parent_dir = "test_images/"

# Path  
path = os.path.join(parent_dir, directory)

try:  
    os.mkdir(path)  
except OSError as error:  
    print(error)


[WinError 183] Cannot create a file when that file already exists: 'test_images/009'


In [2]:
def reconstruction(pc, patient, path): # Depth relates to the mesh quality, the bigger = more details
    poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pc, depth=6, width=0, scale=1.1, linear_fit=False)[0] 
    bbox = pc.get_axis_aligned_bounding_box()
    p_mesh_crop = poisson_mesh.crop(bbox)
    
    filename = patient + '_mesh_poisson.ply'
    o3d.io.write_triangle_mesh(path + '/' + filename, p_mesh_crop)
    
    mesh = pv.read(path + '/' + filename)
    meshfix = mf.MeshFix(mesh)
    meshfix.repair()
    meshfix.mesh.save(path + '/' + filename)
    
    mesh = o3d.io.read_triangle_mesh(path + '/' + filename)
    simple = mesh.filter_smooth_simple(3)
    taubin = simple.filter_smooth_taubin(2) 
    
    o3d.io.write_triangle_mesh(path + '/' + filename, taubin)
    mesh = pv.read(path + '/' + filename)
    
    return mesh

Nube de puntos original

In [3]:
pcd = o3d.io.read_point_cloud(path + "/carlos-rivas_mesh_oriented_original.ply")
print(pcd)
print(np.asarray(pcd.points))
#o3d.visualization.draw_geometries([pcd])


PointCloud with 22986 points.
[[-54.35576762  39.15654645   5.67182065]
 [-55.03675593  34.8927525    8.43971921]
 [-53.46392573  40.72313962  11.03596975]
 ...
 [ 18.65719282 -67.70831027  -3.72476195]
 [ 16.33579313 -68.65908923  -0.90837001]
 [ 16.04756414 -68.65908923  -3.83472346]]


Mesh de entrada

In [4]:
mesh = o3d.io.read_triangle_mesh(path + "/carlos-rivas_mesh_poisson.ply")
mesh.scale(1000, center=mesh.get_center())

T = np.eye(4)
T[:3, :3] = mesh.get_rotation_matrix_from_xyz((100*(np.pi/180),0,np.pi))
print("Matriz T= ",T)

mesh_t = copy.deepcopy(mesh)
mesh_t = mesh_t.transform(T)

Matriz T=  [[-1.00000000e+00 -1.22464680e-16  0.00000000e+00  0.00000000e+00]
 [-2.12657685e-17  1.73648178e-01 -9.84807753e-01  0.00000000e+00]
 [ 1.20604166e-16 -9.84807753e-01 -1.73648178e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [5]:
from scipy.spatial.distance import directed_hausdorff
from scipy.spatial import distance
import math

points=np.asarray(mesh_t.vertices)
pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(points)
pcd2.paint_uniform_color([0, 0, 0.929])

#o3d.visualization.draw_geometries([pcd, pcd3])

trans_matrix = np.eye(4)
reg_p2p = o3d.pipelines.registration.registration_icp(pcd2, pcd, 7000, trans_matrix,o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Matriz de transformacion:")
print(reg_p2p.transformation)
print("")


pcd2.transform(reg_p2p.transformation)

RegistrationResult with fitness=1.000000e+00, inlier_rmse=3.258069e+00, and correspondence_set size of 7950
Access transformation to get result.
Matriz de transformacion:
[[ 0.96968336 -0.04336701  0.24048594 -1.31616257]
 [ 0.05278203  0.99806577 -0.03284487 18.35660041]
 [-0.2385964   0.04454246  0.97009676  0.21153793]
 [ 0.          0.          0.          1.        ]]



PointCloud with 7950 points.

##### Hausdorff

In [6]:
points_t=np.asarray(pcd2.points)
points_or=np.asarray(pcd.points)

#Hausdorff
haus=directed_hausdorff(points_t, points_or)
print('Distancia Hausdorff: ', haus[0])

Distancia Hausdorff:  11.260386741064313


###### DICE

$D = \frac{2⋅|A∩B|}{|A| +|B|} $

$F1 = \frac{2⋅|A∩B|}{1/P + 1/R}  $

$Precision = \frac{|A∩B|}{|A|};  $ $Recall = \frac{|A∩B|}{|B|}  $

$D = F1 = \frac{2}{\frac{|A|}{|A∩B|} + \frac{|B|}{|A∩B|}}$

In [7]:
#DICE
d1 = pcd2.compute_point_cloud_distance(pcd) ##Distancia de cada punto de pcd al pun to mas cercano de pcd
d2 = pcd.compute_point_cloud_distance(pcd2) ##Distancia de cada punto de pcd al pun to mas cercano de pcd2
th=5
print("th=",th)

rec=0
prec=0
for d in d1:
    if d<th:
        prec +=1
for d in d2:
    if d<th:
        rec +=1

if prec>0 and rec>0:
    dice = 2/((len(d1)/prec) + (len(d2)/rec))
else:
    dice=0

print("dice= ", dice)

th= 5
dice=  0.8796233259337747


In [8]:

##### 3D MESH
mesh_t.transform(reg_p2p.transformation)
reconstruction(pcd, 'test', path)
mesh_or = o3d.io.read_triangle_mesh(path + "/test_mesh_poisson.ply")
mesh_or.paint_uniform_color([0.9, 0.706, 0])
o3d.visualization.draw_geometries([mesh_or, mesh_t])
o3d.visualization.draw_geometries([pcd, pcd2])

[Open3D WARNING] Write Ply clamped color value to valid range


Intento de encontrar intersección entre 2 mesh, para calcular DICE.

[PyMesh].(https://pymesh.readthedocs.io/en/latest/mesh_boolean.html#boolean-interface)

[pygalmesh].(https://github.com/nschloe/pygalmesh#domain-combinations)

In [9]:
import meshio
import numpy as np
import pygalmesh
import open3d as o3d
from scipy.spatial.distance import directed_hausdorff

mesh_read1=o3d.io.read_triangle_mesh('carlos-rivas_mesh_poisson.ply')
mesh_read1.scale(1000, center=mesh_read1.get_center())
T = np.eye(4)
T[:3, :3] = mesh_read1.get_rotation_matrix_from_xyz((100*(np.pi/180),0,np.pi))
#T[0, 3] = 1
#T[1, 3] = 1
#T[2, 3] = 1
mesh_read1 = mesh_read1.transform(T)
T2=[[ 0.96971399, -0.04340022,  0.24035639, -1.31605222],
 [ 0.0528083,   0.99806462, -0.03283757, 18.36227267],
 [-0.23846605,  0.04453587,  0.97012911,  0.21510744],
 [ 0.,          0.,          0.,          1.        ]]

mesh_read1 = mesh_read1.transform(T2)
mesh_read2=o3d.io.read_triangle_mesh('test_mesh_poisson.ply')
mesh_read2.paint_uniform_color([1, 0.706, 0])
o3d.visualization.draw_geometries([mesh_read1, mesh_read2])

o3d.io.write_triangle_mesh('uno.ply', mesh_read1)
o3d.io.write_triangle_mesh('dos.ply', mesh_read2)

mesh1=meshio.read('uno.ply')
mesh2=meshio.read('dos.ply')

p=pygalmesh.generate_mesh(mesh1)
p=pygalmesh.generate_mesh(mesh2)

u = pygalmesh.Difference(mesh1, mesh2)

mesh = pygalmesh.generate_mesh(u)
mesh.write('out.stl')
mesh_read= o3d.io.read_triangle_mesh('out.stl')
o3d.visualization.draw_geometries([mesh_read])

ModuleNotFoundError: No module named 'pygalmesh'